## How-to guide for Personalized Promotions use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a model that creates personalized promotions using the Abacus.AI Python Client Library.

We'll be using the [Retail Interaction Logs](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv) and [Item Categories](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv) datasets, which contain information about user interactions and item attributes.

1. Install the Abacus.AI library.

In [ ]:
!pip install abacusai

We'll also import pandas and pprint tools for visualization in this notebook.

In [1]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [2]:
#@title Abacus.AI API Key

api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [3]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [4]:
client.list_use_cases()

[UseCase(use_case='UCPLUGANDPLAY',
   pretty_name='Plug & Play Your Tensorflow Model',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='EMBEDDINGS_ONLY',
   pretty_name='Vector Matching Engine',
   description='Upload embeddings and leverage our similarity search infrastructure.. Scale to high traffic, update your index in near realtime'),
 UseCase(use_case='MODEL_WITH_EMBEDDINGS',
   pretty_name='Tensorflow Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='TORCH_MODEL_WITH_EMBEDDINGS',
   pretty_name='PyTorch Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. H

In this notebook, we're going to create a model that returns personalized promotions using the Retail Interaction Logs and Item Categories datasets. The 'PERS_PROMOTIONS' use case is best tailored for this situation.

In [5]:
#@title Abacus.AI Use Case

use_case = 'PERS_PROMOTIONS'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [6]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

{ 'allowed_feature_mappings': { 'ACTION_TYPE': { 'allowed_feature_types': [ 'CATEGORICAL'],
                                                 'description': 'This is an '
                                                                'optional '
                                                                'column that '
                                                                'specifies the '
                                                                'type of '
                                                                'action the '
                                                                'user took. '
                                                                'This could '
                                                                'include any '
                                                                'action that '
                                                                'is specific '
                                                

Finally, let's create the project.

In [7]:
promotions_project = client.create_project(name='Personalized Promotions Project', use_case=use_case)
promotions_project.to_dict()

{'project_id': '1e58b9d68',
 'name': 'Personalized Promotions Project',
 'use_case': 'PERS_PROMOTIONS',
 'created_at': '2021-11-24T18:54:44+00:00',
 'feature_groups_enabled': True}

**Note: When feature_groups_enabled is False then the use case does not support feature groups (collection of ML features). Therefore, Datasets are created at the organization level and tied to a project to further use them for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using two datasets for this notebook. We'll tell Abacus.AI how each dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Retail Interaction Logs](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv) (**USER_ITEM_INTERACTIONS**): 
This dataset contains information about visitor interactions, with data such as the visitor ID, event, timestamp, etc.
- [Item Categories](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv) (**CATALOG_ATTRIBUTES**): This dataset contains information about items with specified IDs.

### Add the datasets to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [8]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv')

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [9]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv')

,itemid,categoryid
0,19,1171
1,24,1244
2,25,72
3,40,160
4,42,84
...,...,...
59010,466847,1221
59011,466848,1250
59012,466849,464
59013,466858,1191


Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [11]:
user_item_dataset = client.create_dataset_from_file_connector(name='Retail Interaction Logs1',table_name='Retail_Interaction_Logs1',
                                     location='s3://realityengines.datasets/pers_promotion/events.csv',
                                     refresh_schedule='0 12 * * *')

item_attributes_dataset = client.create_dataset_from_file_connector(name='Item Categories1',table_name='Item_Categories1',
                                     location='s3://realityengines.exampledatasets/pers_promotion/item_categories.csv',
                                     refresh_schedule='0 12 * * *')

datasets = [user_item_dataset, item_attributes_dataset]

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [17]:
feature_group = client.create_feature_group(table_name='personalized_promotions',sql='SELECT * from Retail_Interaction_Logs11')

Adding Feature Group to the project:

In [18]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = promotions_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [19]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = promotions_project.project_id, feature_group_type= "USER_ITEM_INTERACTIONS")

Check current Feature Group schema:

In [20]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

[Feature(name='timestamp',
   select_clause=None,
   feature_mapping=None,
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='TIMESTAMP',
   data_type='DATETIME',
   columns=None,
   point_in_time_info=None),
 Feature(name='visitorid',
   select_clause=None,
   feature_mapping=None,
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='event',
   select_clause=None,
   feature_mapping=None,
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='itemid',
   select_clause=None,
   feature_mapping=None,
   so

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [15]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

{'ITEM_ID': {'description': 'This is the unique identifier of each item in your catalog. This is typically your product id, article id, or the video id.',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': True},
 'USER_ID': {'description': 'This is a unique identifier of each user in your user base.',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': True},
 'ACTION_TYPE': {'description': 'This is an optional column that specifies the type of action the user took. This could include any action that is specific to you (e.g., view, click, purchase, rating, comment, like, etc). You can always upload a dataset that has no action_type column if all the actions in the dataset are the same (e.g., a dataset of only purchases or clicks).',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': False},
 'TIMESTAMP': {'description': 'The timestamp when a particular action occurred.',
  'allowed_feature_types': ['TIMESTAMP'],
  'required': False},
 'ACTION_WEIGHT': {'description

In [21]:
client.set_feature_mapping(project_id = promotions_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='itemid',feature_mapping='ITEM_ID')
client.set_feature_mapping(project_id = promotions_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='visitorid',feature_mapping='USER_ID')
client.set_feature_mapping(project_id = promotions_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='timestamp',feature_mapping='TIMESTAMP')

[Feature(name='timestamp',
   select_clause=None,
   feature_mapping='TIMESTAMP',
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='TIMESTAMP',
   data_type='DATETIME',
   columns=None,
   point_in_time_info=None),
 Feature(name='visitorid',
   select_clause=None,
   feature_mapping='USER_ID',
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='event',
   select_clause=None,
   feature_mapping=None,
   source_table='Retail_Interaction_Logs11',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='itemid',
   select_clause=None,
   feature_mapping

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [22]:
client.use_feature_group_for_training(project_id=promotions_project.project_id, feature_group_id=feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [23]:
promotions_project.validate()

ProjectValidation(valid=True,
  dataset_errors=[],
  column_hints={})

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [24]:
promotions_project.get_training_config_options()

[TrainingConfigOptions(name='TEST_SPLIT',
   data_type='INTEGER',
   value_type=None,
   value_options=None,
   value=None,
   default=None,
   options={'range': [5, 20]},
   description='Percent of dataset to use for test data. We support using a range between 6% to 20% of your dataset to use as test data.',
   required=None,
   last_model_value=None),
 TrainingConfigOptions(name='DROPOUT_RATE',
   data_type='INTEGER',
   value_type=None,
   value_options=None,
   value=None,
   default=None,
   options={'range': [0, 90]},
   description='Dropout percentage rate.',
   required=None,
   last_model_value=None),
 TrainingConfigOptions(name='BATCH_SIZE',
   data_type='ENUM',
   value_type=None,
   value_options=None,
   value=None,
   default=None,
   options={'values': [8, 16, 32, 64, 128, 256, 384, 512, 740, 1024]},
   description='Batch size.',
   required=None,
   last_model_value=None),
 TrainingConfigOptions(name='SKIP_HISTORY_FILTERING',
   data_type='BOOLEAN',
   value_type=None,


In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [25]:
promotions_model = promotions_project.train_model(training_config={})
promotions_model.to_dict()

{'name': 'Personalized Promotions Project Model',
 'model_id': '2e8901150',
 'model_config': {},
 'created_at': '2021-11-24T19:26:15+00:00',
 'project_id': '1e58b9d68',
 'shared': False,
 'shared_at': None,
 'train_function_name': None,
 'predict_function_name': None,
 'training_input_tables': None,
 'source_code': None,
 'location': None,
 'refresh_schedules': None,
 'latest_model_version': {'model_version': '9d245170e',
  'status': 'PENDING',
  'model_id': '2e8901150',
  'model_config': {},
  'training_started_at': None,
  'training_completed_at': None,
  'dataset_versions': None,
  'error': None,
  'pending_deployment_ids': None,
  'failed_deployment_ids': None}}

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [26]:
promotions_model.wait_for_evaluation()

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [4]:
# !pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
promotions_project = next(project for project in client.list_projects() if project.name == 'Personalized Promotions Project')
promotions_model = promotions_project.list_models()[-1]
promotions_model.wait_for_evaluation()

Model(name='Personalized Promotions Project Model',
  model_id='2e8901150',
  model_config={},
  created_at='2021-11-24T19:26:15+00:00',
  project_id='1e58b9d68',
  shared=False,
  shared_at=None,
  train_function_name=None,
  predict_function_name=None,
  training_input_tables=None,
  source_code=None,
  location=None,
  refresh_schedules=None,
  latest_model_version=ModelVersion(model_version='9d245170e',
  status='COMPLETE',
  model_id='2e8901150',
  model_config={},
  training_started_at='2021-11-24T19:31:03+00:00',
  training_completed_at='2021-11-24T21:55:59+00:00',
  dataset_versions=['9cb5fcb96'],
  error=None,
  pending_deployment_ids=[],
  failed_deployment_ids=[]))

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics

In [5]:
pp.pprint(promotions_model.get_metrics().to_dict())

{ 'baseline_metrics': None,
  'metrics': { 'coverage': 0.028666666666666667,
               'map': 0.0078014435264858765,
               'map@10': 0.006725259920634921,
               'map@5': 0.006077888888888889,
               'mrr': 0.009566456729611402,
               'ndcg': 0.01694423470519298,
               'ndcg@10': 0.01107502017794698,
               'ndcg@5': 0.0088031855411701,
               'personalization@10': 0.22458635663566143},
  'model_id': '2e8901150',
  'model_version': '9d245170e',
  'target_column': None}


To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/PERS_PROMOTIONS/training) page.

## 4. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [6]:
promotions_deployment = client.create_deployment(name='Personalized Promotions Deployment',model_id=promotions_model.model_id,description='Personalized Promotions Deployment')
promotions_deployment.wait_for_deployment()

Deployment(deployment_id='1428f4669a',
  name='Personalized Promotions Deployment',
  status='ACTIVE',
  description='Personalized Promotions Deployment',
  deployed_at='2021-11-24T22:10:56+00:00',
  created_at='2021-11-24T22:10:17+00:00',
  project_id='1e58b9d68',
  model_id='2e8901150',
  model_version='9d245170e',
  feature_group_id=None,
  feature_group_version=None,
  calls_per_second=5,
  auto_deploy=True,
  regions=[{'name': 'Us East 1', 'value': 'us-east-1'}],
  error=None,
  refresh_schedules=None)

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [7]:
deployment_token = promotions_project.create_deployment_token().deployment_token
deployment_token

'5a8e28cc457842fa97c390eb3e126779'

## 5. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_personalized_ranking` API call below.

This command will return a list with each specified item ID ranked according to the specified visitor's preferences. The ranking would be performed based on what movies the user liked in the past and how the movies and users are related to each other depending on their attributes.

In [8]:
ApiClient().get_personalized_ranking(deployment_token=deployment_token,
                                     deployment_id=promotions_deployment.deployment_id,
                                     query_data={"visitorid":"100018","itemid":["424193","181204","385091","339763","344603"]})

[{'itemid': '181204'},
 {'itemid': '339763'},
 {'itemid': '344603'},
 {'itemid': '385091'},
 {'itemid': '424193'}]